 DOCUMENT PARSER


In [4]:
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
def parseText(pdf_path):
    reader = PdfReader(pdf_path)
    policy_text = ""

    for page_num, page in enumerate(reader.pages, start=1):
        text = page.extract_text()
        if text:
            policy_text += f"\n\n[Page {page_num}]\n{text}"

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=50
    )
    return splitter.split_text(policy_text)

SEMANTIC SEARCH


In [5]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

def build_index(chunks):
    """Build FAISS index for semantic search"""
    embeddings = embed_model.encode(chunks)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    return index

def searchText(question, chunks, index, top_k=3):
    """Retrieve relevant chunks from the policy"""
    q_embedding = embed_model.encode([question])
    _, indices = index.search(q_embedding, top_k)
    return "\n".join([chunks[i] for i in indices[0]])

POLICY DATABASE

In [6]:
class PolicyDatabase:
    """Stores policy chunks and FAISS index"""
    def __init__(self, chunks):
        self.chunks = chunks
        self.index = build_index(chunks)

    def query(self, question):
        return searchText(question, self.chunks, self.index)


In [7]:
def classifyText(question):
    """Simple keyword-based intent classification"""
    question_lower = question.lower()
    if "rent" in question_lower:
        return "room_rent"
    elif "claim" in question_lower:
        return "claims"
    return "general"

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name = "Qwen/Qwen2-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)    

TOOL CALLING CONTROLLER

In [9]:
def tool_router(question):
    """Decide which tool to use based on question"""
    # Here we can integrate classifyText for simplicity
    intent = classifyText(question)
    if intent == "room_rent" or intent == "claims" or intent == "general":
        return "PolicyDatabase.query"
    return "PolicyDatabase.query"

In [24]:
def summarizeText(context, question):
    """Generate a simple, clear answer using the context"""
    prompt = f"""
You are an insurance policy assistant.

STRICT RULES:
- Use ONLY the provided policy context.
- DO NOT assume or guess.
- DO NOT add disclaimers, notes, or recommendations.
- Answer ONLY what is asked.
- If the answer is not explicitly stated, say:
  "The policy does not explicitly mention this information."

Policy Context:
{context}

Question:
{question}

Answer:
"""
    inputs =tokenizer(prompt,return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=120,
        do_sample=False,
        temperature=0.0,
        top_p=1.0
    )

    return tokenizer.decode(output[0],skip_special_tokens=True)

    answer=full_text.split("Answer:")[-1].strip()

    return answer
    

In [25]:
def displayInformation(answer):
    print("\nAnswer:\n")
    print(answer)


In [26]:
if __name__ == "__main__":
    chunks = parseText("policy.pdf")

    policy_db = PolicyDatabase(chunks)

    question = "How much ambulance expense is covered?"

    tool_used = tool_router(question)
    print("Tool Selected by LLM:", tool_used)

    if tool_used == "PolicyDatabase.query":
        context = policy_db.query(question)
    else:
        context = "No relevant tool found."
    answer = summarizeText(context, question)
    displayInformation(answer)


    

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Tool Selected by LLM: PolicyDatabase.query

Answer:


You are an insurance policy assistant.

STRICT RULES:
- Use ONLY the provided policy context.
- DO NOT assume or guess.
- DO NOT add disclaimers, notes, or recommendations.
- Answer ONLY what is asked.
- If the answer is not explicitly stated, say:
  "The policy does not explicitly mention this information."

Policy Context:
8. Ambulance Expenses:  1% of Sum Insured per Policy period up to a maximum of INR 1500 will be reimbursed 
to Insured  for the cost of ambulance transportation.  Ambulance services used should be of a licensed 
ambulance operator. 
 
9. Ayurvedic Medicine: Ayurvedic Treatment covered up to maximum 15% of Sum Insured per Policy Period
restricted maximum up to 25% of the Sum Insured per illness/injury. 
 
2. Medical Practitioner , Surgeon, Anesthetist, Consultants, and Specialists Fees - All admissible claims under 
this section during the policy period restricted maximum up to 40% of the Sum Insured per illness/